In [649]:
#test
print("This notebook is for PGDMLAI Aug.19-Jun.20 - Lei Mao")

This notebook is for PGDMLAI Aug.19-Jun.20 - Lei Mao


In [650]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib, re

In [651]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
#nbayes = MultinomialNB()
#nbayes.fit(sms_messages_train, labels_train)
#accuracy=nbayes.score(sms_messages_test,labels_test)

In [652]:
# import data
rawdata=pd.read_csv('member_sample.csv')  
#check
print(members.info())
print(list(members.columns) )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Data columns (total 63 columns):
Unnamed: 0                      21344 non-null int64
Individual Key                  21344 non-null float64
Household Key                   21344 non-null float64
FSV CMSI Flag                   21344 non-null object
FSV Credit Card Flag            21344 non-null object
FSV Deposit Program Flag        21344 non-null object
FSV Home Equity Flag            21344 non-null object
FSV ID Theft Flag               21344 non-null object
FSV Mortgage Flag               21344 non-null object
INS Client Flag                 21344 non-null object
TRV Globalware Flag             21344 non-null object
Number of Children              12010 non-null object
Length Of Residence             12010 non-null float64
Mail Responder                  12010 non-null object
Home Owner                      11629 non-null object
Income                          11943 non-null object
Children                 

In [653]:
#check nan and remove the the variables that have 50% NAN
#print(members.isna().sum().sort_values(ascending = False))
# Remove fields that not considered useful /explantory variables for this.

In [654]:
list_filter1=['Unnamed: 0', 'Individual Key', 'Household Key', 
               'FSV CMSI Flag', 'FSV Credit Card Flag', 
              'FSV Deposit Program Flag', 'FSV Home Equity Flag',
              'FSV ID Theft Flag', 'FSV Mortgage Flag',
              'INS Client Flag', 'TRV Globalware Flag', 
              'Number of Children', 
              'Length Of Residence', 'Mail Responder',
              'Home Owner', 'Income',  'Children', 
              'Education', 'Dwelling Type', 'Credit Ranges', 
              'Address Change Date',  'Billing Code Description', 
              'Birth Date MMDDYYYY', 'Cancel Date', 
              'Cancel Reason', 'Do Not Direct Mail Solicit',  'ERS ENT Count Year 1',
              'ERS ENT Count Year 2', 'ERS ENT Count Year 3', 
              'ERS Member Cost Year 1', 'ERS Member Cost Year 2', 
              'ERS Member Cost Year 3',  'Join AAA Date', 
              'Join Club Date',
              'Member Status', 'Member Tenure Years', 'Member Type', 
              'Months from Join to Cancel', 
              'Opt-Out - Publication',  'Renew Method', 'ZIP',
               'Mosaic Global Household', 'New Mover Flag', 
               'Occupation Group', 
              'Right_Dwelling Type', 'Move Distance', 
              'Occupant Type',  'Basic Cost',
              'Calculated Tow Miles', 'Call Canceled', 'Call Killed', 
              'SC Date', 'Cash Call',
              'Dispatch Code1 Description', 
              'Plus Cost', 'Plus Indicator Description', 'Premier Cost',
              'SC Vehicle Manufacturer Name', 'SC Vehicle Model Name', 
               'Total Cost']
members=rawdata[list_filter1]

In [655]:
# check member status and type
# considering data filtering

print(members['Member Status'].describe())
print(members['Member Status'].isna().sum())
print(members['Member Status'].value_counts())
#'Mosaic Household', 'Mosaic Global Household'
#print(members['Mosaic Household'].value_counts())

print(members['Member Type'].describe())
print(members['Member Type'].isna().sum())
print(members['Member Type'].value_counts())

count      21343
unique         3
top       ACTIVE
freq       11808
Name: Member Status, dtype: object
1
ACTIVE       11808
CANCELLED     8177
PENDING       1358
Name: Member Status, dtype: int64
count         21343
unique            3
top       Associate
freq          10843
Name: Member Type, dtype: object
1
Associate    10843
Primary      10496
INsider          4
Name: Member Type, dtype: int64


In [656]:
# check to see if data is individual level or record level.
# Data is on service record level, customer profile fields are the same
# for the same individual key
print(members['Individual Key'].describe())
print(members['Individual Key'].isna().sum())
#print(members['Individual Key'].value_counts())
# Individual key 9410475.0 has ~20 records. Export and check.

check1=members[members['Individual Key']==9410475.0]
#check1.to_csv(r'C:\Mark_Mao\MLAL_capstone\check9410475.csv')

count    2.134400e+04
mean     3.358534e+07
std      5.113419e+07
min      1.729300e+04
25%      8.784029e+06
50%      1.853740e+07
75%      2.821761e+07
max      2.123219e+08
Name: Individual Key, dtype: float64
0


In [657]:
# study cancallation

cancelled_members=members[members['Member Status']=="CANCELLED"]
cancelled_members['Cancel Year']=pd.DatetimeIndex(cancelled_members['Cancel Date']).year
#print(cancelled_members['Member Type'].describe())
#print(cancelled_members['Member Type'].isna().sum())
print(cancelled_members['Member Type'].value_counts())
print(cancelled_members['Cancel Reason'].value_counts())
print(cancelled_members['Cancel Year'].value_counts())
# Preliminary Findings 5/12
# 25% are cancelled 2017-2019. 
# Looks like voluntary and involuntary cancellation are 50/50.
# May want to build a model to predict voluntary cancallation to keep
# as much customer as possible which is very important for profits
# Rank order the forecast and actual probability and tag the top 1 or 2 groups to see if we can do anything to keep them
# Any voluntary cancellations can have key words - pay/cancel/ERS/request/renew
# We can remove the involuntary cancellation like "Deceased" & "No longer driving " & "transfer out" & "duplication" &'Data Problem' 



Associate    5688
Primary      2485
INsider         4
Name: Member Type, dtype: int64
Non-Payment                           2680
Deceased                              1640
Members Request                       1550
Data Problem                           465
No Longer Driving                      431
Transfer Member                        332
Cancelled Do Not Renew                 321
Dup Membership                         224
Transfer Member Suspend                183
Moved Out of Terr                       78
Do Not Renew                            76
Cancelled Mbrs Request                  56
No Longer in Household                  41
Cancel Process - Renewal No Pay         12
Health Issue                            12
Noncompliant Age                        12
ERS Cncl MShip Program                  12
Noncompliant Address                     9
ERS Decline CC                           9
Cancel Process - Future Cancel DNR       7
*BAD CHECKS*                             4
Duplicate A

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [658]:
# checking vehicle brand and models
# considering using brand or using models 
# or both (brands + top 50% of models)
# One thing need to do is make sure these fields are all caps to avoid dups
# SC Vehicle Manufacturer Name
#
print(members['SC Vehicle Manufacturer Name'].describe())
print(members['SC Vehicle Manufacturer Name'].isna().sum())
#print(members['SC Vehicle Manufacturer Name'].value_counts())

print(members['SC Vehicle Model Name'].describe())
print(members['SC Vehicle Model Name'].isna().sum())
#print(members['SC Vehicle Model Name'].value_counts())

count      13985
unique        86
top       TOYOTA
freq        2289
Name: SC Vehicle Manufacturer Name, dtype: object
7359
count     13977
unique      788
top       CAMRY
freq        825
Name: SC Vehicle Model Name, dtype: object
7367


In [659]:
print(cancelled_members['SC Vehicle Manufacturer Name'].describe())
print(cancelled_members['SC Vehicle Manufacturer Name'].isna().sum())
#print(members['SC Vehicle Manufacturer Name'].value_counts())

print(cancelled_members['SC Vehicle Model Name'].describe())
print(cancelled_members['SC Vehicle Model Name'].isna().sum())
#print(members['SC Vehicle Model Name'].value_counts())

count       2819
unique        54
top       TOYOTA
freq         425
Name: SC Vehicle Manufacturer Name, dtype: object
5358
count      2816
unique      382
top       CAMRY
freq        161
Name: SC Vehicle Model Name, dtype: object
5361


In [660]:
#comparison among seemingly similar fields:
#  'Clearing Code Last Description'
# -> 78% normal service/ 8% battery /8% canceled call
#  'Dispatch Code1 Description' -> More detailed descrip and more diverfied distribution
#  'DTL Prob1 Code Description' -> Very similar to Dispatch code1.Use first one

#  Picked the top 20 or group issues into various car parts.
#  For the missing, need to assign them as unknown and separate from 
#   other grouped together description
print(members['Dispatch Code1 Description'].describe())
print(members['Dispatch Code1 Description'].isna().sum())
#print(members['Dispatch Code1 Description'].value_counts()) 
    


count                               13996
unique                                 53
top       Member Requests Battery Service
freq                                 3804
Name: Dispatch Code1 Description, dtype: object
7348


In [661]:
#check Total cost from roadside service to see if we need to remove any rows
# check missing - Remove Total cost NAN or missing. 
# Vast majority of records have no values in any roadside service related information
# check very large values 
# One record with $9k cost is collision and membership cancalled due to data problem. Will get removed
# Other Above $ 500 costs are other reasonable or cancalled membership
print(members['Total Cost'].describe())
print(members['Total Cost'].isna().sum())

check_missingcost=members[members['Total Cost'].notnull()==False]
#check_missingcost.to_csv(r'C:\Mark_Mao\MLAL_capstone\check_missingcost.csv')

check_largecost=members[members['Total Cost']>500]
#check_largecost.to_csv(r'C:\Mark_Mao\MLAL_capstone\check_largecost.csv')

count    13944.000000
mean        47.273696
std         86.869110
min          0.000000
25%         30.500000
50%         53.000000
75%         53.000000
max       9869.000000
Name: Total Cost, dtype: float64
7400


In [662]:
# check customer profile - age/education/home
#'Number of Children',-> Group 3&+ together
#'Length Of Residence',-> Group them by int(residence/5)*5 then cap from 0 - 30+
#'Home Owner',-> Overwrite probably into their category for simplicity due to low count.
#'Income', -> May need to combine into buckets to remove some granularity
#'Date Of Birth',->  Half missing from 20-100.Avg.70 for record-level data
# Birth Date MMDDYYYY -> More populated . Use this instead of above.
# 'Education' -> Unknown is one field. Make missing as unknown as well.
#'Dwelling Type' -> Make missing as unknown. 
#'Credit Ranges', -> Make 600 and below as one bucket is good enough.

# Are those missing fields all come from the same member?
# If that is the case, we can call them mysterious members together.

members['Member Age']=2020-pd.DatetimeIndex(members['Birth Date MMDDYYYY']).year

print(members['Member Age'].describe())
#print(members['Credit Ranges'].isna().sum())
#print(members['Credit Ranges'].value_counts()) 

miss_members=members[members['Credit Ranges'].notnull()==False]
miss_members_sample=miss_members.sample(n=200, random_state=100)
#print(miss_members.head(20))
print(miss_members.info())
#miss_members_sample.to_csv(r'C:\Mark_Mao\MLAL_capstone\miss_members_sample.csv')

count    18094.000000
mean        69.462861
std         18.923710
min         18.000000
25%         59.000000
50%         73.000000
75%         83.000000
max        113.000000
Name: Member Age, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9334 entries, 0 to 21340
Data columns (total 61 columns):
Unnamed: 0                      9334 non-null int64
Individual Key                  9334 non-null float64
Household Key                   9334 non-null float64
FSV CMSI Flag                   9334 non-null object
FSV Credit Card Flag            9334 non-null object
FSV Deposit Program Flag        9334 non-null object
FSV Home Equity Flag            9334 non-null object
FSV ID Theft Flag               9334 non-null object
FSV Mortgage Flag               9334 non-null object
INS Client Flag                 9334 non-null object
TRV Globalware Flag             9334 non-null object
Number of Children              0 non-null object
Length Of Residence             0 non-null float6

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [663]:
members['SC Year']=pd.DatetimeIndex(members['SC Date']).year
print(members['SC Year'].value_counts()) 
# Based on autotrader.com 's article:
# Average length of time drivers keep a new vehicle is 71.4 months — around 6 years
# Even though the records goes back to 2014, it may still be good information 
# to forecast next 12 months roadside service cost for members.They probably have the same car.
# Plus big proportion of the data are from pre-2017.

# Data does not tell car brand & model for people who never uses the service so we just use the average. 
# It depends on the variables but we may use 
# Some family income/education to indicate cars they have. It can be taken into account.

# Improvement on data- Provide year /mileage/brand/model for every member and update it frequenctly.

2015.0    3851
2016.0    3551
2014.0    2242
2019.0    2151
2017.0    1160
2018.0    1042
Name: SC Year, dtype: int64


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [664]:
# So far, after 1st round of checking data,we want to remove 
# involuntary cancallations 
# first let's test to see count of individuals level data before and after 1st round of filtering

# need to flag the involuntary cancellations using cancel reasons first
#pay/cancel/ERS/request/renew
members['Cancel Reason']=members['Cancel Reason'].str.lower().astype(str)
members['Voluntary Cancel Flag']=members['Cancel Reason'].str.contains('pay|cancel|ers|request|renew').astype(int)
#members['Voluntary Cancel Flag']=any(ele in voluntary_string for ele in (members['Cancel Reason']))
#print(members['Cancel Reason'].value_counts())
#print(members['Voluntary Cancel Flag'].value_counts())
#print(members.sample(n=20, random_state=100)[['Cancel Reason','Voluntary Cancel Flag']])

filter_members1=members.copy()
filter_members1.drop(filter_members1[(filter_members1['Member Status']=="CANCELLED") & (filter_members1['Voluntary Cancel Flag']==False)].index, inplace=True)
filter_members1.drop(filter_members1[(filter_members1['Individual Key']==10000003.0)].index, inplace=True)
print(filter_members1['Individual Key'].describe())
# Data count goes from 21,344 to 18,126 from 1st round of cleaning on records

print(filter_members1['Individual Key'].nunique())
# individual data count goes at 9,689

print(filter_members1['Household Key'].nunique())
# Household data count goes at 4,748

#filter_members1.to_csv(r'C:\Mark_Mao\MLAL_capstone\filter_members1.csv')

count    1.812500e+04
mean     3.125806e+07
std      4.736165e+07
min      1.989700e+04
25%      8.616479e+06
50%      1.830683e+07
75%      2.799694e+07
max      2.123219e+08
Name: Individual Key, dtype: float64
9688
4747


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [665]:
# Summarize record level data to individual level for modeling
# Cleaning missing values/re-categorizing variables for modeling
 
# Two universal missing flag: Member profile & roadside service history
# Member profile missing flag uses the credit range.
# Roadside service missing flag uses costs
# We can then add more missing flag for specific fields if needed.

filter_members1['m_ProfileMissing']=filter_members1['Credit Ranges'].isnull().astype(int)
filter_members1['m_ServiceMissing']=filter_members1['Total Cost'].isnull().astype(int)

filter_members1.drop(filter_members1[(filter_members1['ERS ENT Count Year 1'].isnull()==True) & (filter_members1['m_ServiceMissing']==0)].index, inplace=True)


# check to see if there is any missing values that need additional flag

# take care of the Tenure Years first
def label_tenure (row):
   if pd.isnull(row['Member Tenure Years']) == False :
      return row['Member Tenure Years']
   if (pd.isnull(row['Member Tenure Years']) == True) & (pd.isnull(row['Join AAA Date']) == False): 
      return 2020-pd.DatetimeIndex(row['Join AAA Date']).year
   if (pd.isnull(row['Member Tenure Years']) == True) & (pd.isnull(row['Join Club Date']) == False): 
      return 2020-pd.DatetimeIndex(row['Join Club Date']).year
   return 0
#combine missing & 0 together as they are both not providing info. Very thin vol.

filter_members1['m_TenureYr'] = filter_members1.apply (lambda row: label_tenure(row), axis=1)

list_profile=['Unnamed: 0', 'Individual Key', 'Household Key', 
              'Number of Children','Length Of Residence',
              'Mail Responder','Home Owner','Income','Credit Ranges',
          'Children','Education','Dwelling Type',
          'Do Not Direct Mail Solicit',
          'Member Type','Opt-Out - Publication','Renew Method',
          'Mosaic Global Household','New Mover Flag',
          'Occupation Group','Right_Dwelling Type','Occupant Type',
          'ZIP','Member Age','Months from Join to Cancel',
              'Member Age','m_ProfileMissing'
]

# will take care of credit range and income separately as they are buckets

list_service=['Unnamed: 0', 'Individual Key', 'Household Key', 
              'Call Canceled','Call Killed','Cash Call',
          'Dispatch Code1 Description','Plus Indicator Description',
          'SC Vehicle Manufacturer Name','SC Vehicle Model Name',
              'Basic Cost','Calculated Tow Miles',
         'Plus Cost','Premier Cost','Total Cost','SC Year',
        'ERS ENT Count Year 1','ERS ENT Count Year 2',
              'ERS ENT Count Year 3','ERS Member Cost Year 1',
              'ERS Member Cost Year 2','ERS Member Cost Year 3'
             ,'m_ServiceMissing' ]

#print(filter_members1['m_TenureYr'].describe())
#print(filter_members1[filter_members1['m_TenureYr'] == 0].shape[0])
#miss_members1=filter_members1[list_profile]
#miss_members2=filter_members1[list_service]

#unmiss_members1check=miss_members1[miss_members1['m_ProfileMissing']==0]
#unmiss_members2check=miss_members2[miss_members2['m_servicemissing']==0]
#unmiss_members1check.to_csv(r'C:\Mark_Mao\MLAL_capstone\unmiss_members1.csv')
#unmiss_members2check.to_csv(r'C:\Mark_Mao\MLAL_capstone\unmiss_members2.csv')

# Observation from the csv file: Needed the missing flags
# Home Owner / Occupant Type / Mosaic Global Household / Occupation Group /
# Dwelling Type /Right_Dwelling Type / Months from Join to Cancel /Member Age

filter_members1['MosaicMissing']=filter_members1['Mosaic Global Household'].isnull().astype(int)
filter_members1['m_OccupationMissing']=filter_members1['Occupation Group'].isnull().astype(int)
filter_members1['m_MosJoin2CancelMissing']=filter_members1['Months from Join to Cancel'].isnull().astype(int)
filter_members1['m_AgeMissing']=filter_members1['Months from Join to Cancel'].isnull().astype(int)
filter_members1['m_IncomeMissing']=filter_members1['Income'].isnull().astype(int)
filter_members1['m_CreditMissing']=filter_members1['Credit Ranges'].isnull().astype(int)


######## Home Owner/Renter/ Age ###############
def label_homeowner (row):
   if row['Home Owner'] == 'Home Owner' :
      return 1
   if row['Home Owner']  == 'Probable Home Owner' : 
      return 1
   if row['Occupant Type'] == 'HomeOwner':
      return 1
   return 0

def label_renter (row):
   if row['Home Owner'] == 'Renter' :
      return 1
   if row['Home Owner']  == 'Probable Renter' : 
      return 1
   if row['Occupant Type'] == 'Renter':
      return 1
   return 0

filter_members1['m_HomeownerFlag'] = filter_members1.apply (lambda row: label_homeowner(row), axis=1)
filter_members1['m_RenterFlag'] = filter_members1.apply (lambda row: label_renter(row), axis=1)
filter_members1['m_memberage'] =filter_members1['Member Age']


# Dwelling Type /Right_Dwelling Type
# First use Right_Dwelling to help populate missing in dwelling type
# Dwelling type has much less missing and is seen as main field
filter_members1['Dwelling Type'].fillna(filter_members1['Right_Dwelling Type'])

def label_dwell (row):
   if row['Dwelling Type'] == 'Small or large multi-family w/apt number' :
      return 'MFDU'
   if row['Dwelling Type']  == 'Multi Family With' : 
      return 'MFDU'
   if row['Dwelling Type']  == 'Large multi-family w/o apt number' : 
      return 'MFDU'
   if row['Dwelling Type']  == 'Multi Family No' : 
      return 'MFDU'
   if row['Dwelling Type'] == 'SFDU':
      return 'SFDU'
   if row['Dwelling Type']  == 'Single Family' : 
      return 'SFDU'
   if row['Dwelling Type'] == 'P.O. BOX':
      return 'POBOX'
   if row['Dwelling Type']  == 'PO Box' : 
      return 'POBOX'
   return 'Unknown'
filter_members1['m_Dwelling'] = filter_members1.apply (lambda row: label_dwell(row), axis=1)
filter_members1['m_DwellMissing']=filter_members1['Dwelling Type'].isnull().astype(int)

# WIll not use 'Call killed' - All "N" for nonmissing.
# one missing for Dispatch Code1 Description - make it unknown
# Will categorize the description later.
# A few missing - SC Vehicle Model Name - will not use directly
# A few missing - Calculated Tow Miles - Set to 0 for simplicity.
# A few missing - Premier Cost - Set to 0 
# Will use total cost - basic cost = plus & premier.if <0 then use plus and premier.
# There are 29 have missing count and costs per year1-3. Dropped in the codes above.



In [666]:
 #########   # of child /len of resid /mail respond  #####################

# majority of # of childs are below 3. Group 3+ all together as one.
def label_nchild (row):
   if row['Number of Children'] == 'No children' :
      return 0
   if row['Number of Children']  == 'One Child' : 
      return 1
   if row['Number of Children'] == 'Two Children' :
      return 2
   if row['Number of Children'] == 'Three Children' :
      return 3
   if row['Number of Children'] == 'Four Children' :
      return 4
   if row['Number of Children']  == 'Unknown':
      return 0
   return 5 # 5 means 5+

filter_members1['m_nChild'] = filter_members1.apply (lambda row: label_nchild(row), axis=1)
filter_members1['m_LenResidence'] =filter_members1.apply(lambda row: max([min([row['Length Of Residence'], 20]),0]), axis=1)
filter_members1['m_MailRespond'] = filter_members1['Mail Responder'] 

#print(filter_members1['m_nChild'].value_counts())


In [667]:
# After missing flags are created, populate those missing values 

filter_members1.fillna({'Mosaic Global Household':'Unknown',
                 'Occupation Group':'Unknown',
                 'Dispatch Code1 Description':'Unknown',
                 'Income':'Unknown',
                 'Credit Ranges':'Unknown'}, inplace=True)

filter_members1.fillna(0,inplace = True) 
#filter_members1.fillna('Unknown',inplace = True) 

In [668]:
#checking some fields

#print(filter_members1['Income'].value_counts())
#print(filter_members1['Income'].isna().sum())
#print(filter_members1['Credit Ranges'].value_counts())
#print(filter_members1['Credit Ranges'].isna().sum())
#print(filter_members1['m_memberage'].describe())
#print(filter_members1['m_memberage'].isna().sum())
#print(filter_members1['Opt-Out - Publication'].value_counts())
# Checked that they are populated as expected

#print(filter_members1['Move Distance'].value_counts())

# not very useful with so many missing

In [669]:
# Part 2                      
#'m_Income',
def label_income (row):
   if row['Income'] == 'Under 10K' :
      return 10000
   if row['Income'] == '10-19,999' :
      return 10000
   if row['Income'] == '20-29,999' :
      return 30000
   if row['Income'] == '30-39,999' :
      return 30000
   if row['Income'] == '40-49,999' :
      return 50000
   if row['Income'] == '50-59,999' :
      return 50000
   if row['Income'] == '60-69,999' :
      return 70000
   if row['Income'] == '70-79,999' :
      return 70000
   if row['Income'] == '80-89,999' :
      return 90000
   if row['Income'] == '90-99,999' :
      return 90000
   if row['Income'] == '100-149,999' :
      return 125000
   if row['Income'] == '150 - 174,999':
      return 167500
   if row['Income'] == '175 - 199,999' :
      return 187500
   if row['Income'] == '200 - 249,999' :
      return 225000
   if row['Income'] == '250K+' :
      return 250000
   return 999 # set missing/unknown to 999                          
filter_members1['m_Income'] = filter_members1.apply (lambda row: label_income(row), axis=1)
         
#'m_Credit'
def label_credit (row):
   if row['Credit Ranges'] == '499 & Less' :
      return 500
   if row['Credit Ranges'] == '500-549' :
      return 500
   if row['Credit Ranges'] == '550-599' :
      return 550
   if row['Credit Ranges'] == '600-649' :
      return 600
   if row['Credit Ranges'] == '650-699' :
      return 650
   if row['Credit Ranges'] == '700-749' :
      return 700
   if row['Credit Ranges'] == '750-799' :
      return 750
   if row['Credit Ranges'] == '800+' :
      return 800
   return 999 # set missing/unknown to 999
filter_members1['m_Credit'] = filter_members1.apply (lambda row: label_credit(row), axis=1)

In [670]:
#print(filter_members1['m_Income'].value_counts())

In [671]:
#Part 3:Create some more binary variables
filter_members1['m_MailRespondYes'] = filter_members1['m_MailRespond'].apply(lambda x: 1 if x == 'Yes' else 0)
filter_members1['m_NoChild']=filter_members1['Children'].apply(lambda x: 1 if x == 'No' else 0)
filter_members1['m_No2DirectMail']=filter_members1['Do Not Direct Mail Solicit'].apply(lambda x: 1 if x == 'No' else 0)
filter_members1['m_PrimaryMember']=filter_members1['Member Type'].apply(lambda x: 1 if x == 'Primary' else 0)
filter_members1['m_OptIn_Publication']=filter_members1['Opt-Out - Publication'].apply(lambda x: 1 if x == 'Opt-In' else 0)
filter_members1['m_CashCallYes'] = filter_members1['Cash Call'].apply(lambda x: 1 if x == 'Y' else 0)
filter_members1['m_MosaicMIssing'] = filter_members1.apply(lambda row: (row['m_ProfileMissing'])*(row['MosaicMissing'] ), axis = 1)


In [672]:
#New Mover Flag + address change flag
# New Mover flag not consistent with address change date.
# Will use both
# If address change year is later than or equal to 2017. Consider new mover too.
def label_move (row):
   if row['New Mover Flag'] == 'Y' :
      return 1
   if (pd.isnull(row['New Mover Flag']) == True) & (pd.DatetimeIndex([row['Address Change Date']]).year>=2017) :
      return 1
   if (row['New Mover Flag'] == 'N') & (pd.DatetimeIndex([row['Address Change Date']]).year>=2017) :
      return 1
   return 0 # set missing/unknown to 0
filter_members1['m_NewMover'] = filter_members1.apply (lambda row: label_move(row), axis=1)


# Plus and Premier are combined together as higher level membership compared to basic
def label_plus (row):
   if row['Plus Indicator Description'] == 'Plus Membership' :
      return 1
   if row['Plus Indicator Description'] == 'Plus Membership with Motorcycle Coverage' :
      return 1
   if row['Plus Indicator Description'] == 'Premier Membership' :
      return 1    
   return 0 # set missing/unknown to 0
filter_members1['m_PlusMember'] = filter_members1.apply (lambda row: label_plus(row), axis=1)


In [673]:
# clean up occupation group for easy dummy creation

def label_occ (row):
   if row['Occupation Group'] == 'Blue Collar - Known' :
      return 'Blue_Collar'
   if row['Occupation Group'] == 'Blue Collar - Inferred  ' :
      return 'Blue_Collar'
   if row['Occupation Group'] == 'Management - Known' :
      return 'Management'   
   if row['Occupation Group'] == 'Management - Inferred' :
      return 'Management'     
   if row['Occupation Group'] == 'Office Administration - Known' :
      return 'OfficeAdmin'
   if row['Occupation Group'] == 'Office Administration - Inferred' :
      return 'OfficeAdmin'  
   if row['Occupation Group'] == 'Professional - Known' :
      return 'Professional'
   if row['Occupation Group'] == 'Professional - Inferred' :
      return 'Professional'
   if row['Occupation Group'] == 'Retired - Known' :
      return 'Retired'    
   if row['Occupation Group'] == 'Retired - Inferred' :
      return 'Retired' 
   if row['Occupation Group'] == 'Sales - Known' :
      return 'Sales'
   if row['Occupation Group'] == 'Sales - Inferred' :
      return 'Sales'
   if row['Occupation Group'] == 'Technical - Known' :
      return 'Technical'
   if row['Occupation Group'] == 'Technical - Inferred' :
      return 'Technical'
   return 'Other' # set missing/unknown to other
filter_members1['m_OccGroup'] = filter_members1.apply (lambda row: label_occ(row), axis=1)


In [674]:
# Use Mosaic Global Household for the member that have a lot of missing info.
# due to the limited understanding of that segmentation from external source
# Let's create a new field that links to Mosaic segment if profile is missing
def label_mosaic (row):
   if (row['m_ProfileMissing'] == 1) & (row['MosaicMissing'] == 0 ) :
      return row['Mosaic Global Household']   
   return 'Unknown' # set missing/unknown to 0
filter_members1['m_Mosaic'] = filter_members1.apply (lambda row: label_mosaic(row), axis=1)


In [675]:
# check to make sure fields are created as expected
#filter_members1[['m_TenureYr','Member Tenure Years',  'Join AAA Date', 'Join Club Date']].sample(n=50)
#filter_members1[['m_PlusMember','Plus Indicator Description']].sample(n=50, random_state=100)
#filter_members1[['m_NewMover','New Mover Flag','Address Change Date']].sample(n=50, random_state=100)
#filter_members1[['m_OccGroup','Occupation Group']].sample(n=50, random_state=100)

In [676]:
# create dummy variables 
dummy=pd.get_dummies(filter_members1['m_OccGroup'], prefix='Occ', prefix_sep='_', dummy_na=False)
filter_members1=filter_members1.merge(dummy,left_index=True,right_index=True)

dummy1=pd.get_dummies(filter_members1['Education'], prefix='Ed', prefix_sep='_', dummy_na=False)
filter_members1=filter_members1.merge(dummy1,left_index=True,right_index=True)

dummy2=pd.get_dummies(filter_members1['m_Mosaic'], prefix='Mosaic', prefix_sep='_', dummy_na=False)
filter_members1=filter_members1.merge(dummy2,left_index=True,right_index=True)

# Forgot about renew method - add flag binary variable here. Majority are "Notice"
filter_members1['m_AutoRenew'] = filter_members1['Renew Method'].apply(lambda x: 1 if x == 'AUTO RENEW' else 0)


In [677]:
dummy3=pd.get_dummies(filter_members1['m_Dwelling'], prefix='Dwell', prefix_sep='_', dummy_na=False)
filter_members1=filter_members1.merge(dummy3,left_index=True,right_index=True)


In [678]:
#filter_members1.head()
#filter_members1.tail(10)
print(filter_members1.describe())

         Unnamed: 0  Individual Key  Household Key  Length Of Residence  \
count  18096.000000    1.809600e+04   1.809600e+04         18096.000000   
mean   29255.952586    3.125696e+07   1.501145e+07             7.230603   
std    26266.883092    4.736795e+07   1.968149e+07             7.082909   
min        1.000000    1.989700e+04   8.750000e+02             0.000000   
25%     6824.750000    8.616464e+06   4.297359e+06             0.000000   
50%    19353.500000    1.830679e+07   9.747512e+06             7.000000   
75%    51393.250000    2.799658e+07   1.607606e+07            15.000000   
max    99998.000000    2.123219e+08   1.000791e+08            49.000000   

       Do Not Direct Mail Solicit  ERS ENT Count Year 1  ERS ENT Count Year 2  \
count                18096.000000           18096.00000          18096.000000   
mean                     0.047027               0.42700              0.792164   
std                      0.211702               0.91373              1.204976   


In [679]:
# Create luxury car and sport car flag based on SC car and model;
# RLX|A6|A7|A8|Q5|Q7|528|530|535|740|750|760|X5|CT6|CTS|ESCALADE|XTS|TAHOE|SUBURBAN|YUKON|GS3|GX4|LS4|LX4|AVIATOR|CONTINENTAL|NAVIGATOR|E3|E4|E5|GL|AMG|S3|S4|S5|S6|SEQUOIA|LAND CRUISER|TOUAREG|S80|S90
filter_members1['m_ExpensiveModel']=filter_members1['SC Vehicle Model Name'].str.contains('RLX|A6|A7|A8|Q5|Q7|528|530|535|740|750|760|X5|CT6|CTS|ESCALADE|XTS|TAHOE|SUBURBAN|YUKON|GS3|GX4|LS4|LX4|AVIATOR|CONTINENTAL|NAVIGATOR|E3|E4|E5|GL|AMG|S3|S4|S5|S6|SEQUOIA|LAND CRUISER|TOUAREG|S80|S90')

# S4|TT|Z3|Z4|CAMARO|CORVETTE|MUSTANG|MIATA|RX-7|RX-8|SL5|SLK5|BRZ|WRX|911|912|914|944|BOXSTER|
filter_members1['m_SportsCar']=filter_members1['SC Vehicle Model Name'].str.contains('S4|Z3|Z4|CAMARO|CORVETTE|MUSTANG|MIATA|RX-7|RX-8|SL5|SLK5|BRZ|WRX|911|912|914|944|BOXSTER')

#SC Vehicle Manufacturer Name

# For auto manufacturer - categorize into 7 groups for easier modeling
# Japan| Japan Luxury | European | European Luxury | American |American Luxury | Other
def label_auto (row):
   if row['SC Vehicle Manufacturer Name'] == 'HONDA' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'MAZDA' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'MITSUBISHI' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'NISSAN' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'SUBARU' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'TOYOTA' :
      return 'Japan'
   if row['SC Vehicle Manufacturer Name'] == 'ACURA' :
      return 'Japan_Luxury'  
   if row['SC Vehicle Manufacturer Name'] == 'INFINITI' :
      return 'Japan_Luxury'      
   if row['SC Vehicle Manufacturer Name'] == 'LEXUS' :
      return 'Japan_Luxury'      
   if row['SC Vehicle Manufacturer Name'] == 'MINI' :
      return 'European'   
   if row['SC Vehicle Manufacturer Name'] == 'VOLKSWAGEN' :
      return 'European' 
   if row['SC Vehicle Manufacturer Name'] == 'ALFA ROMEO' :
      return 'European_Luxury'
   if row['SC Vehicle Manufacturer Name'] == 'AUDI' :
      return 'European_Luxury'    
   if row['SC Vehicle Manufacturer Name'] == 'BMW' :
      return 'European_Luxury'        
   if row['SC Vehicle Manufacturer Name'] == 'JAGUAR' :
      return 'European_Luxury'     
   if row['SC Vehicle Manufacturer Name'] == 'LAND ROVER' :
      return 'European_Luxury'        
   if row['SC Vehicle Manufacturer Name'] == 'MERCEDES-BENZ' :
      return 'European_Luxury'     
   if row['SC Vehicle Manufacturer Name'] == 'VOLVO' :
      return 'European_Luxury'      
   if row['SC Vehicle Manufacturer Name'] == 'BUICK' :
      return 'American'   
   if row['SC Vehicle Manufacturer Name'] == 'CHEVROLET' :
      return 'American'     
   if row['SC Vehicle Manufacturer Name'] == 'CHRYSLER' :
      return 'American'    
   if row['SC Vehicle Manufacturer Name'] == 'DODGE' :
      return 'American'        
   if row['SC Vehicle Manufacturer Name'] == 'FORD' :
      return 'American' 
   if row['SC Vehicle Manufacturer Name'] == 'GMC' :
      return 'American' 
   if row['SC Vehicle Manufacturer Name'] == 'JEEP' :
      return 'American'     
   if row['SC Vehicle Manufacturer Name'] == 'RAM' :
      return 'American'    
   if row['SC Vehicle Manufacturer Name'] == 'CADILLAC' :
      return 'American_Luxury'
   if row['SC Vehicle Manufacturer Name'] == 'LINCOLN' :
      return 'American_Luxury'
   return 'Other' # set missing/unknown to Other

filter_members1['m_CarBrand'] = filter_members1.apply (lambda row: label_auto(row), axis=1)

filter_members1.fillna({'m_SportsCar':False,'m_ExpensiveModel':False,
                 'm_CarBrand':'Other'}, inplace=True)

filter_members1['m_SportsCar']=filter_members1['m_SportsCar'].astype(int)
filter_members1['m_ExpensiveModel']=filter_members1['m_ExpensiveModel'].astype(int)

print(filter_members1[['m_SportsCar','m_ExpensiveModel','SC Vehicle Model Name','SC Vehicle Manufacturer Name','m_CarBrand']].sample(n=20))


       m_SportsCar  m_ExpensiveModel SC Vehicle Model Name  \
20767            0                 1                 TAHOE   
8915             0                 0                     0   
2834             0                 0                SONATA   
5270             0                 0                     0   
18487            0                 0              SPORTAGE   
7027             0                 0                     0   
42               0                 0                     0   
16462            0                 0                     0   
14201            0                 0                     0   
12772            0                 0                     0   
9586             0                 0               ELANTRA   
7748             0                 0                  XC70   
17183            0                 0                VERANO   
4134             0                 0                     0   
20157            0                 0                  CR-V   
11952   

In [680]:
# create dummy for car brand
dummycar=pd.get_dummies(filter_members1['m_CarBrand'], prefix='Car', prefix_sep='_', dummy_na=False)
filter_members1=filter_members1.merge(dummycar,left_index=True,right_index=True)


In [681]:
# Roadside servicer: the average anuual # of usage and average annual cost
# for the past 3 years.
# decided to not use the total cost/basic cost/... due to messy quality.
filter_members1['m_Avg_ERS_ENT_Count_PerYr'] = filter_members1.apply(lambda row: ((row['ERS ENT Count Year 1'])+(row['ERS ENT Count Year 2']) +(row['ERS ENT Count Year 3'])/3), axis = 1)
filter_members1['m_Avg_ERS_Cost_PerYr'] = filter_members1.apply(lambda row: ((row[ 'ERS Member Cost Year 1'])+(row[ 'ERS Member Cost Year 2']) +(row[ 'ERS Member Cost Year 3'])/3), axis = 1)


def label_prENT (row):
   if row['m_Avg_ERS_ENT_Count_PerYr'] > 0 :
      return 1  
   return 0 # set missing/unknown to 0
filter_members1['m_Prob_ERS_ENT_Used'] = filter_members1.apply (lambda row: label_prENT(row), axis=1)

# record level data do help identify the cars household are driving.

#print(filter_members1[['m_Prob_ERS_ENT_Used','m_Avg_ERS_ENT_Count_PerYr','ERS ENT Count Year 1','ERS ENT Count Year 2','ERS ENT Count Year 3']].sample(n=20))
#print(filter_members1[['m_Avg_ERS_Cost_PerYr','ERS Member Cost Year 1','ERS Member Cost Year 2','ERS Member Cost Year 3']].sample(n=20))


In [682]:
# Check the complete list of fields at this stage 
# and separate into member profile & road service data
# Then merge them later
# list(filter_members1.columns.values)

In [683]:
# For modeling - non-roadside service part
# Summarize this to the individual level and output it 

Fullset_indi=['Individual Key','Household Key','Voluntary Cancel Flag',
              'm_PlusMember','FSV CMSI Flag','FSV Credit Card Flag',
              'FSV Deposit Program Flag','FSV Home Equity Flag',
              'FSV ID Theft Flag','FSV Mortgage Flag','INS Client Flag',
              'TRV Globalware Flag','Months from Join to Cancel',
              'm_ProfileMissing','m_TenureYr','m_OccupationMissing',
              'm_MosJoin2CancelMissing','m_AgeMissing','m_IncomeMissing',
              'm_CreditMissing','m_HomeownerFlag','m_RenterFlag',
              'm_memberage','m_DwellMissing','m_nChild','m_LenResidence',
              'm_MailRespond','m_Income','m_Credit','m_MailRespondYes',
              'm_NoChild','m_No2DirectMail','m_PrimaryMember',
              'm_OptIn_Publication','m_CashCallYes','m_MosaicMIssing',
              'm_NewMover','Occ_Blue_Collar','Occ_Management',
              'Occ_OfficeAdmin','Occ_Other','Occ_Professional',
              'Occ_Retired','Occ_Sales','Occ_Technical',
              'Ed_Completed College','Ed_Graduated School',
              'Ed_High School','Ed_Some College',
              'Mosaic_Bourgeois Prosperity','Mosaic_Career and Family',
              'Mosaic_Comfortable Retirement',
              'Mosaic_Hard Working Blue Collar','Mosaic_Low Income Elders',
              'Mosaic_Metropolitan Strugglers',
              'Mosaic_Post Industrial Survivors',
              'Mosaic_Routine Service Workers','Mosaic_Rural Inheritance',
              'Mosaic_Sophisticated Singles','m_AutoRenew','Dwell_MFDU',
              'Dwell_POBOX','Dwell_SFDU']

Profile_fullset=filter_members1[Fullset_indi]

Profile_fullset = Profile_fullset.sort_values('Individual Key', ascending=False)
Profile_fullset = Profile_fullset.drop_duplicates(subset='Individual Key', keep='first')
Profile_activeset=Profile_fullset[Profile_fullset['Voluntary Cancel Flag']==0]

Profile_fullset.to_csv(r'C:\Mark_Mao\MLAL_capstone\CleanSet\Profile_fullset_noDup.csv')
Profile_activeset.to_csv(r'C:\Mark_Mao\MLAL_capstone\CleanSet\Profile_activeset_noDup.csv')

In [684]:
print(filter_members1.head())

   Unnamed: 0  Individual Key  Household Key FSV CMSI Flag  \
1           1      52211550.0      4500791.0             N   
2           2      52211550.0      4500791.0             N   
3           3      52211550.0      4500791.0             N   
4           4      52211550.0      4500791.0             N   
5           5      52211550.0      4500791.0             N   

  FSV Credit Card Flag FSV Deposit Program Flag FSV Home Equity Flag  \
1                    Y                        N                    N   
2                    Y                        N                    N   
3                    Y                        N                    N   
4                    Y                        N                    N   
5                    Y                        N                    N   

  FSV ID Theft Flag FSV Mortgage Flag INS Client Flag  ... Car_American  \
1                 N                 N               N  ...            0   
2                 N                 N       

In [692]:
# # For modeling - Roadside service part
# Summarize this to the Household level and output it 
# Probability of ERS ENT usage uses all the data

# need to separate into by household by car model set to sum up the # of cars.
# need to separate into by household by individual set to avg./sum up member info.
# for target need to sum up by household by records (all)

# Average cost and count uses only the data that Usage=Yes only.

# It will be a two stage model.
# One is simply the probability of using.
# Then Probability x Average cost gives expected annual cost
# probability x average count gives expected annual count

Fullset_house_nodup_record=[ 'Household Key','m_Avg_ERS_ENT_Count_PerYr',
                             'm_Avg_ERS_Cost_PerYr','m_Prob_ERS_ENT_Used']

Fullset_house_nodup_record1=[ 'm_Avg_ERS_ENT_Count_PerYr',
                             'm_Avg_ERS_Cost_PerYr','m_Prob_ERS_ENT_Used']


Fullset_house_forSUM_car=[ 'Household Key', 'SC Vehicle Model Name','m_ExpensiveModel',
                          'm_SportsCar', 'Car_American', 'Car_American_Luxury', 
                          'Car_European', 'Car_European_Luxury', 'Car_Japan',
                          'Car_Japan_Luxury', 'Car_Other']

Fullset_house_forSUM_car1=['m_ExpensiveModel',
                          'm_SportsCar', 'Car_American', 'Car_American_Luxury', 
                          'Car_European', 'Car_European_Luxury', 'Car_Japan',
                          'Car_Japan_Luxury', 'Car_Other']

Fullset_house_forAVG_indi=[ 'Individual Key','Household Key','m_PlusMember', 'Occ_Blue_Collar', 
                          'Occ_Management', 'Occ_OfficeAdmin', 'Occ_Other',
                          'Occ_Professional', 'Occ_Retired', 'Occ_Sales', 'Occ_Technical',
                          'Ed_Completed College', 'Ed_Graduated School', 'Ed_High School', 
                          'Ed_Some College', 'm_ProfileMissing', 'm_TenureYr',
                          'm_OccupationMissing', 'm_MosJoin2CancelMissing', 'm_AgeMissing',
                          'm_IncomeMissing', 'm_CreditMissing', 'm_HomeownerFlag', 
                          'm_RenterFlag', 'm_memberage', 'm_DwellMissing', 'm_nChild', 
                          'm_LenResidence', 'm_MailRespond', 'm_Income', 'm_Credit', 
                          'm_MailRespondYes', 'm_NoChild', 'm_No2DirectMail', 
                          'm_PrimaryMember', 'm_OptIn_Publication', 'm_CashCallYes',
                          'm_MosaicMIssing', 'Mosaic_Bourgeois Prosperity', 
                          'Mosaic_Career and Family', 'Mosaic_Comfortable Retirement',
                          'Mosaic_Hard Working Blue Collar', 'Mosaic_Low Income Elders',
                          'Mosaic_Metropolitan Strugglers', 'Mosaic_Post Industrial Survivors',
                          'Mosaic_Routine Service Workers', 'Mosaic_Rural Inheritance', 
                          'Mosaic_Sophisticated Singles', 'm_AutoRenew', 'Dwell_MFDU', 
                          'Dwell_POBOX', 'Dwell_SFDU', 'm_NewMover'
              ]

Fullset_house_forAVG_indi1=[ 'Occ_Blue_Collar', 'm_PlusMember',
                          'Occ_Management', 'Occ_OfficeAdmin', 'Occ_Other',
                          'Occ_Professional', 'Occ_Retired', 'Occ_Sales', 'Occ_Technical',
                          'Ed_Completed College', 'Ed_Graduated School', 'Ed_High School', 
                          'Ed_Some College', 'm_ProfileMissing', 'm_TenureYr',
                          'm_OccupationMissing', 'm_MosJoin2CancelMissing', 'm_AgeMissing',
                          'm_IncomeMissing', 'm_CreditMissing', 'm_HomeownerFlag', 
                          'm_RenterFlag', 'm_memberage', 'm_DwellMissing', 'm_nChild', 
                          'm_LenResidence', 'm_MailRespond', 'm_Income', 'm_Credit', 
                          'm_MailRespondYes', 'm_NoChild', 'm_No2DirectMail', 
                          'm_PrimaryMember', 'm_OptIn_Publication', 'm_CashCallYes',
                          'm_MosaicMIssing', 'Mosaic_Bourgeois Prosperity', 
                          'Mosaic_Career and Family', 'Mosaic_Comfortable Retirement',
                          'Mosaic_Hard Working Blue Collar', 'Mosaic_Low Income Elders',
                          'Mosaic_Metropolitan Strugglers', 'Mosaic_Post Industrial Survivors',
                          'Mosaic_Routine Service Workers', 'Mosaic_Rural Inheritance', 
                          'Mosaic_Sophisticated Singles', 'm_AutoRenew', 'Dwell_MFDU', 
                          'Dwell_POBOX', 'Dwell_SFDU', 'm_NewMover'
              ]


#roadside data only uses active member data. Cancelled data may be
# out of date
filter_members2=filter_members1[filter_members1['Voluntary Cancel Flag']==0]

########### for costs/count  just remove duplicates on household level ##########
RoadService_nodup=filter_members2[Fullset_house_nodup_record]
RoadService_nodup.sort_values('Household Key', inplace=True) 
RoadService_nodup=pd.pivot_table(data=RoadService_nodup, index='Household Key', values=Fullset_house_nodup_record1, aggfunc='mean')
RoadService_forSUM_car.reset_index()

########### for car information remove dupliecate on household + car level then sum ##########
# only downside is that one family can have two of the same models 
RoadService_forSUM_car=filter_members2[Fullset_house_forSUM_car]
RoadService_forSUM_car.sort_values(['Household Key', 'SC Vehicle Model Name']) 
RoadService_forSUM_car.drop_duplicates(subset =['Household Key', 'SC Vehicle Model Name'],keep='first', inplace=True)
RoadService_forSUM_car.drop(['SC Vehicle Model Name'], axis=1)
RoadService_forSUM_car2=pd.pivot_table(data=RoadService_forSUM_car, index='Household Key', values=Fullset_house_forSUM_car1, aggfunc='sum')
RoadService_forSUM_car2.reset_index()

#print(RoadService_forSUM_car.head())

########### for profile information remove dupliecate on household + individual level then avg ##########
RoadService_forAVG_indi=filter_members2[Fullset_house_forAVG_indi]
RoadService_forAVG_indi.sort_values(['Household Key', 'Individual Key']) 
RoadService_forAVG_indi.drop_duplicates(subset =['Household Key','Individual Key'],keep='first', inplace=True)
RoadService_forAVG_indi.drop(['Individual Key'], axis=1)
RoadService_forAVG_indi2=pd.pivot_table(data=RoadService_forAVG_indi, index='Household Key', values=Fullset_house_forAVG_indi1, aggfunc='mean')
RoadService_forSUM_car2.reset_index()

RoadService_nodup = RoadService_nodup.sort_values('Household Key', ascending=False)
RoadService_forSUM_car2 = RoadService_forSUM_car2.sort_values('Household Key', ascending=False)
RoadService_forAVG_indi2 = RoadService_forAVG_indi2.sort_values('Household Key', ascending=False)

#merge 
RoadServicefull_nodup=RoadService_nodup.merge(RoadService_forSUM_car2,on='Household Key',how='left')
RoadServicefull_nodup2=RoadServicefull_nodup.merge(RoadService_forAVG_indi2,on='Household Key',how='left')
RoadServicefull_nodup2.fillna(0)
#check

RoadServicefull_nodup2.to_csv(r'C:\Mark_Mao\MLAL_capstone\CleanSet\RoadService_activeset_noDup.csv')
RoadService_forSUM_car2.to_csv(r'C:\Mark_Mao\MLAL_capstone\CleanSet\RoadService_check.csv')


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [686]:
#filter_members2.to_csv(r'C:\Mark_Mao\MLAL_capstone\CleanSet\filter_member2.csv')